<a href="https://colab.research.google.com/github/ethanweed/praat-workshop/blob/main/advanced_vowelspace_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Plotting vowel spaces


<img src="https://upload.wikimedia.org/wikipedia/commons/8/87/Vowel_triange%2C_cardinal_vowels.png" width="400"/>

image credit: Kwamikagami (wikipedia.org)


Let's compare different speakers producing the vowels in English words 'heed', 'hid', 'head', 'had', 'hod', 'hawed', 'hood', and 'who'd'.

<img src="https://kagi.com/proxy/images?c=_m3km2RjA3G0qleowsZXHZb9NEn0fSsEYIHbKzMDyAFb4nUPIanknmQV_g0rmdCIbMWuvyo7994ui97cT-3EedvKHC3hiNCl3h5OMx-XVM9HyXvCWU_IGx_mPDPr1dfEaGFA-mOvqORHh0X-9LrKmg%3D%3D" width="400"/>

image credit: www.phonetics.ucla.edu

<img src="https://kagi.com/proxy/tongue.png?c=VyuR3YRKUClPhyOoIdS04TqljqgBzrKDxski68jlhLPReVad1H0Zm0Kgd4UwEzxC-foBCJc2-1ghyNdX1b6XRg%3D%3D" width="400"/>

image credit: Wilfred Bates



# Plot vowels

Let's use Python to plot our extracted formant vowels and inspect our vowels spaces.
We'll start with a very simple figure, where we simply plot the raw F1 and F2 frequencies for the cardinal vowels.
Things to notice:
1. We need to import libraries
2. We use `pandas` to import and structure our data
3. We use `seaborn` to make plotting easier
4. We use `matplotlib.pyplot` to make adjustments to our figure
5. We rename libraries with abreviations

In [ ]:
# import data structuring and plotting libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# quiet warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Read our txt file into a pandas dataframe
#df = pd.read_csv('https://raw.githubusercontent.com/ethanweed/praat-workshop/refs/heads/main/2025/output/h_d_formants/combined_formants.txt', sep='\t')
df = pd.read_csv('https://raw.githubusercontent.com/ethanweed/praat-workshop/refs/heads/main/2025/output/cardinal_vowels_formants/combined_formants.txt', sep='\t')

# inspect the first 5 lines
df

In [ ]:
ax = sns.scatterplot(data=df, x=df['F2'], y=df['F1'])
ax.invert_yaxis()
ax.invert_xaxis()

## Fancier vowel plots using helper functions

The vowelplot, barkify, and Lobify functions below were written by [Emily Remirez](https://sites.google.com/berkeley.edu/eremirez/) and copied from: [Linguistics Methods Hub: Vowel plotting in Python](https://lingmethodshub.github.io/content/python/vowel-plotting-py/index.html).

Using Emily's functions, we can quickly and easily make professional-looking vowel plots for comparing speakers, dialects, or languages.


In [ ]:
# vowelplot function
# written by Emily Remirez
# small modification to remove left and right spines by Ethan Weed

import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame
import seaborn as sns
import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter

def vowelplot (vowelcsv, color=None, F1="F1", F2="F2", vowel="Vowel", title="Vowel Plot", unit="Hz", logscale=True):

    #Set some parameters for the chart itself
    sns.set(style='ticks', context='notebook')
    plt.figure(figsize=(6,6))

    # If there's an argument for color, determine whether it's likely to be categorical
    ## If it's a string (text), use a categorical color palette
    ## If it's a number, use a sequential color palette
    if color != None:
        if type(vowelcsv[color].iloc[0])==str:
            pal = "husl"
            #pal = "Dark2"
        else:
            pal = "viridis"

        pl = sns.scatterplot(x = F2,
                             y = F1,
                             hue = color,
                             data = vowelcsv,
                             palette = pal)

    # If no color argument is given, don't specify hue, and no palette needed
    else:
        pl = sns.scatterplot(x = F2,
                             y = F1,
                             data = vowelcsv)


    #Invert axes to correlate with articulatory space!
    pl.invert_yaxis()
    pl.invert_xaxis()

    #Add unit to the axis labels
    F1name = str("F1 ("+unit+")")
    F2name = str("F2 ("+unit+")")
    laby = plt.ylabel(F1name)
    labx = plt.xlabel(F2name)

    if logscale == True:
        pl.loglog()
        pl.yaxis.set_major_formatter(ticker.ScalarFormatter())
        pl.yaxis.set_minor_formatter(ticker.ScalarFormatter())
        pl.xaxis.set_major_formatter(ticker.ScalarFormatter())
        pl.xaxis.set_minor_formatter(ticker.ScalarFormatter())

    # Add vowel labels

    if vowel != None:
        for line,row in vowelcsv.iterrows():
            pl.text(vowelcsv[F2][line]+0.1,
                    vowelcsv[F1][line],
                    vowelcsv[vowel][line],
                    horizontalalignment = 'left',
                    size = 14, # Edit for larger plots!
                    color = 'black',
                   # weight = 'semibold' # Uncomment for larger plots!
                   )

    pl.set_title(title)
    pl.spines['top'].set_visible(False)
    pl.spines['right'].set_visible(False)
    plt.show()

    return pl

In [ ]:
# function to convert formants from Hz to bark units
# a bark scale is a better approximation of our nonliear pyschoacoustic perception of the physical sound frequencies

def barkify (data, formants):
    # For each formant listed, make a copy of the column prefixed with z
    for formant in formants:
        for ch in formant:
            if ch.isnumeric():
                num = ch
        formantchar = (formant.split(num)[0])
        name = str(formant).replace(formantchar,'z')
        # Convert each value from Hz to Bark
        data[name] = 26.81/ (1+ 1960/data[formant]) - 0.53
    # Return the dataframe with the changes
    return data

In [ ]:
# a function to normalize formants based on each speaker's vowel space, using the Lobanov method.
# for details and descriptions, including pros and cons for each method, as well as an online tool for using the different algorithms, see http://lingtools.uoregon.edu/norm/norm1_methods.php

def Lobify (data, group, formants):
    zscore = lambda x: (x - x.mean()) / x.std()
    for formant in formants:
        name = str("zsc_" + formant)
        col = data.groupby([group])[formant].transform(zscore)
        data.insert(len(data.columns), name, col)
    return data


Load the data into a `pandas` dataframe

In [ ]:
# load the data into a dataframe
df = pd.read_csv('https://raw.githubusercontent.com/ethanweed/praat-workshop/refs/heads/main/2025/output/h_d_formants/combined_formants.txt', sep='\t')

#labels = ['heed', 'hid', 'head', 'had', 'hod', 'hawed', 'hood', 'whod']
labels = ['i', 'I', 'ɛ', 'æ', 'ɑ', 'ɔ', 'ʊ', 'u']

# Make a new column called speaker with the string bewten "h_d_" and "_formants" in Source_File
df['Speaker'] = df['Source_File'].str.extract(r'(h_d_)(.*)(_formants)')[1]

label_dict = {
    'Segment_1': 'i',
    'Segment_2': 'I',
    'Segment_3': 'ɛ',
    'Segment_4': 'æ',
    'Segment_5': 'ɑ',
    'Segment_6': 'ɔ',
    'Segment_7': 'ʊ',
    'Segment_8': 'u'
}

# Make a column called vowel. Look up the key in the Label column and return the value from the label_dict
df['Vowel'] = df['Label'].map(label_dict)

# Make a new column called dialect with the string in speaker removing any numbers
df['Dialect'] = df['Speaker'].str.extract(r'([a-zA-Z]+)')[0]

df


In [ ]:
vowelplot(df, color = 'Dialect')

In [ ]:
barkify(df, ["F1","F2"])

In [ ]:
vowelplot(df,
          F1 = "z1",
          F2 = "z2",
          color = "Dialect",
          unit = "Bark",
          logscale = False)

In [ ]:
Lobify(df,
       group = "Speaker",
       formants = ["F1","F2"]
      )

In [ ]:
vowelplot(df,
          F1 = "zsc_F1",
          F2 = "zsc_F2",
          color = "Dialect",
          unit = "Lobanov normalized from Bark",
          logscale = False)
